In [1]:
!pip install uv
!uv pip install -q autogluon.timeseries --system


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
error: Failed to prepare distributions
  Caused by: Failed to download and build `numpy==1.22.4`
  Caused by: Build backend failed to determine requirements with `build_wheel()` (exit status: 1)

[stderr]
Traceback (most recent call last):
  File "<string>", line 8, in <module>
  File "/home/yl2672496l/.cache/uv/builds-v0/.tmpTrHZYU/lib/python3.12/site-packages/setuptools/__init__.py", line 10, in <module>
    import distutils.core
ModuleNotFoundError: No module named 'distutils'
  Caused by: distutils was removed from the standard library in Python 3.12. Consider adding a constraint (like `numpy >1.22.4`) to avoid building a version of numpy that depends on distutils.


In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np

# Read data

In [ ]:
my_data = pd.read_csv('GD030A_S.csv')

In [ ]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [ ]:
traffic_full = recover_timestamp(my_data)
traffic_full

In [ ]:
train_set = traffic_full[:'2022-02-28 23:00:00']
valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

In [ ]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates sequences for time series data, excluding any sequences containing NaN values.

    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.

    Returns:
    - sequences_df: pandas DataFrame containing all sequences with input lags and output leads,
                    and an 'item_id' column to separate each sequence, including timestamps.
    """
    sequences = []
    item_id = 0  # To assign unique ID to each sequence

    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow'].values
        X_timestamps = data.iloc[i - input_length:i].index
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow'].values
        y_timestamps = data.iloc[i:i + forecast_horizon].index

        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            # Convert X_seq and y_seq to DataFrames, including their timestamps
            x_df = pd.DataFrame({'target': X_seq, 'timestamp': X_timestamps})
            y_df = pd.DataFrame({'target': y_seq, 'timestamp': y_timestamps})

            # Concatenate X_df and y_df
            seq_df = pd.concat([x_df, y_df], ignore_index=True)

            # Add 'item_id' column
            seq_df['item_id'] = str(item_id)
            item_id += 1  # Increment item_id

            # Append to list
            sequences.append(seq_df)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs

    # Concatenate all sequences into one DataFrame
    sequences_df = pd.concat(sequences)
    sequences_df['item_id'] = sequences_df['item_id'].astype('str')
    sequences_df['target'] = sequences_df['target'].astype('float32')

    return sequences_df

In [ ]:
input_lengths = [24 * i for i in range(1, 22)]
prediction_length = 6

In [ ]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")

    # Create sequences with forecast_horizon=6
    test = create_sequences(test_set, length, forecast_horizon=6)

    # Store in the dictionary
    data_dict[length]['test'] = test

    # Print shapes and ensure no NaNs
    print(f"  test shape: {test.index.max()+1}, {len(test.item_id.unique())}\n")

# Zero-shot

In [ ]:
def chronos_zero_shot(input_lengths, prediction_length, model_type):
    results_dict = defaultdict(dict)

    for length in input_lengths:
        print(f"Processing input length: {length}")

        # prepare the data
        test_data = TimeSeriesDataFrame(data_dict[length]['test'])
        test_data_ready, test_data = test_data.train_test_split(prediction_length)

        predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(test_data_ready, presets=model_type)
        predictions = predictor.predict(test_data_ready)

        ground_truth_df = test_data.groupby(level='item_id').tail(6)
        predicted_df = predictions[['mean']]

        results_dict[length]['Predicted_Results'] = predicted_df
        results_dict[length]['Gound_Truth'] = ground_truth_df

    return results_dict

In [ ]:
results_dict = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")

In [ ]:
def cal_metrics(df_true, df_pred):
    df_merged = pd.merge(df_true, df_pred, on=['item_id', 'timestamp'], how='inner')

    # Step 3: Assign prediction step numbers (1 to 6) within each 'item_id'
    df_merged['step'] = df_merged.groupby('item_id').cumcount() + 1

    # Step 4: Calculate error metrics
    df_merged['error'] = df_merged['mean'] - df_merged['target']
    df_merged['abs_error'] = df_merged['error'].abs()
    df_merged['squared_error'] = df_merged['error'] ** 2

    # Handle division by zero in MAPE calculation by adding a very small number (epsilon)
    epsilon = 1e-10
    df_merged['abs_percentage_error'] = df_merged['abs_error'] / (df_merged['target'].abs() + epsilon)

    # Step 5: Calculate overall metrics
    overall_mse = df_merged['squared_error'].mean()
    overall_rmse = np.sqrt(overall_mse)
    overall_mae = df_merged['abs_error'].mean()
    overall_mape = df_merged['abs_percentage_error'].mean() * 100

    print("Overall Evaluation Metrics:")
    print(f"MSE: {overall_mse:.4f}")
    print(f"RMSE: {overall_rmse:.4f}")
    print(f"MAE: {overall_mae:.4f}")
    print(f"MAPE: {overall_mape:.2f}%")

    # Step 6: Calculate metrics for each prediction step
    metrics_by_step = df_merged.groupby('step').agg({
      'squared_error': 'mean',
      'abs_error': 'mean',
      'abs_percentage_error': 'mean'
    }).reset_index()

    metrics_by_step['MSE'] = metrics_by_step['squared_error']
    metrics_by_step['RMSE'] = np.sqrt(metrics_by_step['squared_error'])
    metrics_by_step['MAE'] = metrics_by_step['abs_error']
    metrics_by_step['MAPE'] = metrics_by_step['abs_percentage_error'] * 100

    # Select relevant columns and round for better readability
    metrics_by_step = metrics_by_step[['step', 'MSE', 'RMSE', 'MAE', 'MAPE']]

    return metrics_by_step

In [ ]:
for length in input_lengths:
    print(f"Processing input length: {length}")

    predict = results_dict[length]['Predicted_Results']
    truth = results_dict[length]['Gound_Truth']
    evaluation_df = cal_metrics(predict, truth)
    display(evaluation_df)